# This is a sample Jupyter Notebook

Below is an example of a code cell.
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import os, requests
import random
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import numpy as np
import math

from google.auth import api_key

In [14]:
load_dotenv()
API_KEY = os.environ.get('key')
REGION = 'na1'
root = 'https://americas.api.riotgames.com'
winRateDict = {}
pickRateDict = {}
top5WR = {}

champList = ['aphelios', 'ashe', 'caitlyn', 'corki', 'draven', 'ezreal', 'jhin', 'jinx', 'kaisa', 'kalista', 'kogmaw', 'lucian', 'missfortune', 'nilah', 'samira', 'sivir', 'smolder', 'tristana', 'twitch', 'varus', 'vayne', 'xayah', 'zeri']
champListLength = len(champList)
headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; YourAppName/1.0)'
}

In [31]:
def updateWinRates(winRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            winRateEm = soup.find('em', string='Win rate')
            winRateB = winRateEm.find_next('b', class_='text-[12px]')
            if winRateB:
                raw_text = winRateB.get_text(separator=" ", strip=True)
                winRate = raw_text.replace('"', '').strip()
                winRateDict.update({champ: float(winRate.strip(" %"))})
        else:
            print(f"Failed to retrieve data: {response.status_code}")

def updatePickRates(pickRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            pickRateEm = soup.find('em', string='Pick rate')
            pickRateB = pickRateEm.find_next('b', class_='text-[12px]')
            if pickRateB:
                raw_text = pickRateB.get_text(separator=" ", strip=True)
                pickRate = raw_text.replace('"', '').strip()
                pickRateDict.update({champ: float(pickRate.strip(" %"))})
        else:
            print(f"Failed to retrieve data: {response.status_code}")

def getTop5(top5, winRateDict):
    for key, value in winRateDict.items():
        currWR = float(winRateDict.get(key))
        if len(top5) < 5:
            top5[key] = currWR
        else:
            smallest_key = min(top5, key=top5.get)
            if currWR > top5[smallest_key]:
                top5.pop(smallest_key)
                top5[key] = currWR

def addZScores(dictionary):
    zScoreDict = {}
    std = np.std(list(dictionary.values()))
    mean = np.mean(list(dictionary.values()))
    for key, value in dictionary.items():
        floatVal = float(value)
        currentKey = key
        zScore = float((floatVal - mean) / std)
        zScoreDict[key] = zScore
    return zScoreDict

def joinZPicKRate(winRateDict, zscoreDict):
    result = {}
    for key, value in winRateDict.items():
        result.update({key: 1.5 * value + 2 * zscoreDict[key]})
    return result

def getPUUID(username, tagline):
    response = requests.get(f'{root}//riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={API_KEY}')
    return response.json()['puuid']

def getMatchHistory(count, username, tagline):
    matches = requests.get(f'{root}/lol/match/v5/matches/by-puuid/{playerPUUID}/ids?start=0&count={count}&api_key={API_KEY}').json()
    return matches

def getMatchInformation(matchID):
    return requests.get(f'{root}/lol/match/v5/matches/{matchID}?api_key={API_KEY}').json()

In [6]:
updateWinRates(winRateDict, headers)
updatePickRates(pickRateDict, headers)

In [191]:
getTop5(top5WR, winRateDict)

In [7]:
zScorePRDict = addZScores(pickRateDict)
zScoreWRDict = addZScores(winRateDict)

In [8]:
truePowerLevelDict = joinZPicKRate(zScoreWRDict, zScorePRDict)

top5PL = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True)[:5])
truePowerLevelDict = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True))
sortedWR = dict(sorted(winRateDict.items(), key=lambda item: item[1], reverse=True))
sortedPR = dict(sorted(pickRateDict.items(), key=lambda item: item[1], reverse=True))

print(f'Win Rates: {sortedWR}\n')
print(f'Pick Rates: {sortedPR}\n')
print(f'Power Level: {truePowerLevelDict}\n')
print(f'Top 5: {top5PL}\n')

Win Rates: {'nilah': 53.65, 'kogmaw': 53.38, 'vayne': 52.3, 'jinx': 52.02, 'tristana': 51.85, 'missfortune': 51.39, 'twitch': 51.34, 'lucian': 51.25, 'sivir': 51.1, 'samira': 50.75, 'xayah': 50.69, 'jhin': 50.64, 'zeri': 50.57, 'draven': 50.51, 'corki': 50.48, 'ashe': 50.37, 'smolder': 50.24, 'aphelios': 50.14, 'kaisa': 49.75, 'caitlyn': 49.74, 'varus': 49.64, 'kalista': 48.61, 'ezreal': 47.86}

Pick Rates: {'kaisa': 21.37, 'ezreal': 19.04, 'jhin': 19.04, 'jinx': 17.14, 'caitlyn': 12.5, 'lucian': 11.33, 'tristana': 10.96, 'ashe': 9.07, 'varus': 7.41, 'missfortune': 6.95, 'vayne': 6.11, 'aphelios': 5.6, 'smolder': 5.43, 'xayah': 5.08, 'draven': 4.77, 'twitch': 4.26, 'sivir': 4.23, 'corki': 3.51, 'zeri': 3.5, 'samira': 3.47, 'kogmaw': 2.0, 'nilah': 2.0, 'kalista': 1.86}

Power Level: {'jinx': 4.511332741876502, 'jhin': 3.568722353460782, 'kaisa': 3.339650385610109, 'tristana': 2.19546681627814, 'lucian': 1.6292179147203698, 'nilah': 1.2019996329036462, 'vayne': 1.0519817723065195, 'kogma

In [45]:
try:
    username, tagline = input("Enter username and tagline (e.g. Saico#Slide): ").split("#")
    print(f"Username: {username}, Tagline: {tagline}")
except ValueError:
    print("Invalid entry method, try again.\n")

Username: Saico, Tagline: Slide


In [68]:
playerPUUID = getPUUID(username, tagline)
count = 3

matchList = getMatchHistory(count, username, tagline)

['NA1_5255639416', 'NA1_5255620896', 'NA1_5255598057']


In [73]:
for game in matchList:
    info = game['info']
    queueID = info['queueId'] #Ranked is 420
    if int(queueID) != 420:
        break
    metadata = game['metadata']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(playerPUUID)]
    playerTeam = player['teamId']
    matchID = metadata['matchId']



16
1700
420
10
